## drop fnlwgt
- It's a Census sampling weight, not a personal characteristic
- Near-zero correlation with income (-0.0095)
- Would not be available for new individuals we want to predict

## education_num — is it truly ordinal / proper label encoding?
Let's look at the mapping:
Preschool=1, 1st-4th=2, 5th-6th=3, 7th-8th=4, 9th=5, 10th=6,
11th=7, 12th=8, HS-grad=9, Some-college=10, Assoc-voc=11,
Assoc-acdm=12, Bachelors=13, Masters=14, Prof-school=15, Doctorate=16
This is ordinal — it follows a logical progression of educational attainment. And the numeric spacing is roughly meaningful: each step represents "more education." So treating it as a numeric feature is reasonable.
However, it's not a perfect linear scale. Is the jump from HS-grad (9) to Some-college (10) the same as Masters (14) to Prof-school (15)? Probably not in terms of income impact. So it's a reasonable label encoding but with the caveat that the relationship with income may not be perfectly linear.
For tree-based models (Random Forest, XGBoost) this doesn't matter — they handle ordinal features natively. For linear models, we might want to consider binning or treating it as categorical.
My recommendation: keep education_num as-is, drop education. It's good enough for most models and has the strongest correlation with income (0.335).

## binary transformations
marital_status → Married / Not-married: The >50K rate splits cleanly — Married-civ-spouse (45%) and Married-AF-spouse (44%) vs everything else (4-10%). The signal is really about being married vs not. Simplifying reduces dimensionality from 7 categories to 1 binary.
native_country → US / Non-US: 90% of the data is US. With 41 categories, most have tiny sample sizes and unreliable >50K rates. Binary encoding captures the main signal without noise.
income → binary 1/0: Necessary for modeling, straightforward.
capital_gain → has_capital_gain: 92% zeros. The presence of capital gains is a strong wealth signal. We could also keep the original amount as a second feature (log-transformed) for those who have it.
capital_loss → has_capital_loss: Same logic, 95% zeros.

Question for you on capital_gain/loss: should we keep only the binary flag, or also keep the original amount (maybe log-transformed) alongside it? The amount might carry additional signal — e.g., someone with 99,999 capital gain is very different from someone with 500.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set_style("whitegrid")

# Load data
df = pd.read_csv(r"C:\Users\andras.janko\Documents\CensusIncomePrediction\_data\adult.csv")

# Clean column names
df.columns = df.columns.str.replace('.', '_', regex=False)

# Strip whitespace from string columns
str_cols = df.select_dtypes(include='object').columns
df[str_cols] = df[str_cols].apply(lambda c: c.str.strip())

# Replace '?' with NaN
df.replace('?', np.nan, inplace=True)

print(f"Raw shape: {df.shape}")
df.head()

Raw shape: (32561, 15)


,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,90,NaN,77053,HS-grad,9,Widowed,NaN,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,NaN,186061,Some-college,10,Widowed,NaN,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [2]:
# 1. Drop fnlwgt (sampling weight, no predictive value)
# 2. Drop education (redundant with education_num)
# 3. Drop race (imbalanced, ethical concerns, modest signal)
df.drop(columns=['fnlwgt', 'education', 'race'], inplace=True)

# 4. Binary: income target (>50K = 1, <=50K = 0)
df['income'] = (df['income'] == '>50K').astype(int)

# 5. Binary: marital_status (Married vs Not-married)
#    Married-civ-spouse and Married-AF-spouse → 1, rest → 0
df['marital_status'] = df['marital_status'].isin(['Married-civ-spouse', 'Married-AF-spouse']).astype(int)

# 6. Binary: native_country (US vs Non-US)
df['native_country'] = (df['native_country'] == 'United-States').astype(int)

# 7. Log transform: capital_gain and capital_loss (log1p to handle zeros)
df['capital_gain'] = np.log1p(df['capital_gain'])
df['capital_loss'] = np.log1p(df['capital_loss'])

# 8. Fill missing values in workclass and occupation with 'Unknown'
df['workclass'].fillna('Unknown', inplace=True)
df['occupation'].fillna('Unknown', inplace=True)

print(f"Processed shape: {df.shape}")
print(f"\nColumn types:\n{df.dtypes}")
print(f"\nMissing values: {df.isnull().sum().sum()}")
print(f"\nTarget distribution:\n{df['income'].value_counts()}")
print(f"\nFirst rows:")
df.head()

Processed shape: (32561, 12)

Column types:
age                 int64
workclass          object
education_num       int64
marital_status      int64
occupation         object
relationship       object
sex                object
capital_gain      float64
capital_loss      float64
hours_per_week      int64
native_country      int64
income              int64
dtype: object

Missing values: 0

Target distribution:
income
0    24720
1     7841
Name: count, dtype: int64

First rows:


C:\Users\andras.janko\AppData\Local\Temp\ipykernel_22144\3613449587.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['workclass'].fillna('Unknown', inplace=True)
C:\Users\andras.janko\AppData\Local\Temp\ipykernel_22144\3613449587.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a

,age,workclass,education_num,marital_status,occupation,relationship,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,90,Unknown,9,0,Unknown,Not-in-family,Female,0.0,8.379539,40,1,0
1,82,Private,9,0,Exec-managerial,Not-in-family,Female,0.0,8.379539,18,1,0
2,66,Unknown,10,0,Unknown,Unmarried,Female,0.0,8.379539,40,1,0
3,54,Private,4,0,Machine-op-inspct,Unmarried,Female,0.0,8.268988,40,1,0
4,41,Private,10,0,Prof-specialty,Own-child,Female,0.0,8.268988,40,1,0


In [3]:
# Fix fillna using proper pandas syntax
df['workclass'] = df['workclass'].fillna('Unknown')
df['occupation'] = df['occupation'].fillna('Unknown')

print(f"Missing values: {df.isnull().sum().sum()}")
print(f"\nworkclass value counts (top 5):")
print(df['workclass'].value_counts().head())
print(f"\noccupation value counts (top 5):")
print(df['occupation'].value_counts().head())

Missing values: 0

workclass value counts (top 5):
workclass
Private             22696
Self-emp-not-inc     2541
Local-gov            2093
Unknown              1836
State-gov            1298
Name: count, dtype: int64

occupation value counts (top 5):
occupation
Prof-specialty     4140
Craft-repair       4099
Exec-managerial    4066
Adm-clerical       3770
Sales              3650
Name: count, dtype: int64


In [4]:
cat_cols = df.select_dtypes(include='object').columns.tolist()

print(f"Remaining categorical columns: {cat_cols}\n")
for col in cat_cols:
    print(f"--- {col} ({df[col].nunique()} unique) ---")
    print(df[col].value_counts())
    print()

Remaining categorical columns: ['workclass', 'occupation', 'relationship', 'sex']

--- workclass (9 unique) ---
workclass
Private             22696
Self-emp-not-inc     2541
Local-gov            2093
Unknown              1836
State-gov            1298
Self-emp-inc         1116
Federal-gov           960
Without-pay            14
Never-worked            7
Name: count, dtype: int64

--- occupation (15 unique) ---
occupation
Prof-specialty       4140
Craft-repair         4099
Exec-managerial      4066
Adm-clerical         3770
Sales                3650
Other-service        3295
Machine-op-inspct    2002
Unknown              1843
Transport-moving     1597
Handlers-cleaners    1370
Farming-fishing       994
Tech-support          928
Protective-serv       649
Priv-house-serv       149
Armed-Forces            9
Name: count, dtype: int64

--- relationship (6 unique) ---
relationship
Husband           13193
Not-in-family      8305
Own-child          5068
Unmarried          3446
Wife             

In [5]:
# How much does relationship overlap with marital_status + sex?
print("=== relationship vs marital_status + sex ===")
print(pd.crosstab([df['marital_status'], df['sex']], df['relationship'], margins=True))

=== relationship vs marital_status + sex ===
relationship           Husband  Not-in-family  Other-relative  Own-child  \
marital_status sex                                                         
0              Female        0           3870             376       2200   
               Male          0           4418             480       2772   
1              Female        1              5              54         45   
               Male      13192             12              71         51   
All                      13193           8305             981       5068   

relationship           Unmarried  Wife    All  
marital_status sex                             
0              Female       2654     0   9100  
               Male          792     0   8462  
1              Female          0  1566   1671  
               Male            0     2  13328  
All                         3446  1568  32561  


In [6]:
# First, merge Never-worked and Without-pay into Unknown
df['workclass'] = df['workclass'].replace({'Never-worked': 'Unknown', 'Without-pay': 'Unknown'})

print(f"workclass unique after merge: {df['workclass'].nunique()}")
print(f"occupation unique: {df['occupation'].nunique()}")
print(f"\nOne-hot would add: {df['workclass'].nunique() + df['occupation'].nunique() - 2} columns")
print(f"Target encoding would add: 2 columns")

# Preview what target encoding would look like
print("\n=== Target encoding: mean income by workclass ===")
print(df.groupby('workclass')['income'].mean().round(4).sort_values(ascending=False))

print("\n=== Target encoding: mean income by occupation ===")
print(df.groupby('occupation')['income'].mean().round(4).sort_values(ascending=False))

# Check if the target-encoded values have good spread
print(f"\nWorkclass target-enc range: {df.groupby('workclass')['income'].mean().min():.4f} - {df.groupby('workclass')['income'].mean().max():.4f}")
print(f"Occupation target-enc range: {df.groupby('occupation')['income'].mean().min():.4f} - {df.groupby('occupation')['income'].mean().max():.4f}")

workclass unique after merge: 7
occupation unique: 15

One-hot would add: 20 columns
Target encoding would add: 2 columns

=== Target encoding: mean income by workclass ===
workclass
Self-emp-inc        0.5573
Federal-gov         0.3865
Local-gov           0.2948
Self-emp-not-inc    0.2849
State-gov           0.2720
Private             0.2187
Unknown             0.1029
Name: income, dtype: float64

=== Target encoding: mean income by occupation ===
occupation
Exec-managerial      0.4840
Prof-specialty       0.4490
Protective-serv      0.3251
Tech-support         0.3050
Sales                0.2693
Craft-repair         0.2266
Transport-moving     0.2004
Adm-clerical         0.1345
Machine-op-inspct    0.1249
Farming-fishing      0.1157
Armed-Forces         0.1111
Unknown              0.1036
Handlers-cleaners    0.0628
Other-service        0.0416
Priv-house-serv      0.0067
Name: income, dtype: float64

Workclass target-enc range: 0.1029 - 0.5573
Occupation target-enc range: 0.0067 - 0.48

In [7]:
# Drop relationship (redundant with marital_status + sex)
df.drop(columns=['relationship'], inplace=True)

# Encode sex as binary
df['sex'] = (df['sex'] == 'Male').astype(int)

# One-hot encode workclass and occupation (drop_first to avoid multicollinearity)
df = pd.get_dummies(df, columns=['workclass', 'occupation'], drop_first=True)

print(f"Final shape: {df.shape}")
print(f"\nAll columns:\n{list(df.columns)}")
print(f"\nData types:\n{df.dtypes}")
print(f"\nSample:")
df.head()

Final shape: (32561, 29)

All columns:
['age', 'education_num', 'marital_status', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'income', 'workclass_Local-gov', 'workclass_Private', 'workclass_Self-emp-inc', 'workclass_Self-emp-not-inc', 'workclass_State-gov', 'workclass_Unknown', 'occupation_Armed-Forces', 'occupation_Craft-repair', 'occupation_Exec-managerial', 'occupation_Farming-fishing', 'occupation_Handlers-cleaners', 'occupation_Machine-op-inspct', 'occupation_Other-service', 'occupation_Priv-house-serv', 'occupation_Prof-specialty', 'occupation_Protective-serv', 'occupation_Sales', 'occupation_Tech-support', 'occupation_Transport-moving', 'occupation_Unknown']

Data types:
age                               int64
education_num                     int64
marital_status                    int64
sex                               int64
capital_gain                    float64
capital_loss                    float64
hours_per_week                    int64
n

,age,education_num,marital_status,sex,capital_gain,capital_loss,hours_per_week,native_country,income,workclass_Local-gov,...,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,occupation_Unknown
0,90,9,0,0,0.0,8.379539,40,1,0,False,...,False,False,False,False,False,False,False,False,False,True
1,82,9,0,0,0.0,8.379539,18,1,0,False,...,False,False,False,False,False,False,False,False,False,False
2,66,10,0,0,0.0,8.379539,40,1,0,False,...,False,False,False,False,False,False,False,False,False,True
3,54,4,0,0,0.0,8.268988,40,1,0,False,...,False,True,False,False,False,False,False,False,False,False
4,41,10,0,0,0.0,8.268988,40,1,0,False,...,False,False,False,False,True,False,False,False,False,False


In [8]:
# Convert bool columns to int
bool_cols = df.select_dtypes(include='bool').columns
df[bool_cols] = df[bool_cols].astype(int)

# Final validation
print(f"Shape: {df.shape}")
print(f"Missing values: {df.isnull().sum().sum()}")
print(f"All numeric: {df.select_dtypes(include='number').shape[1] == df.shape[1]}")
print(f"\nTarget distribution:\n{df['income'].value_counts()}")
print(f"\n=== Final describe ===")
df.describe().round(3)

Shape: (32561, 29)
Missing values: 0
All numeric: True

Target distribution:
income
0    24720
1     7841
Name: count, dtype: int64

=== Final describe ===


,age,education_num,marital_status,sex,capital_gain,capital_loss,hours_per_week,native_country,income,workclass_Local-gov,...,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,occupation_Unknown
count,32561.000,32561.000,32561.000,32561.000,32561.000,32561.000,32561.000,32561.000,32561.000,32561.000,...,32561.000,32561.000,32561.000,32561.000,32561.000,32561.00,32561.000,32561.000,32561.000,32561.000
mean,38.582,10.081,0.461,0.669,0.735,0.350,40.437,0.896,0.241,0.064,...,0.042,0.061,0.101,0.005,0.127,0.02,0.112,0.029,0.049,0.057
std,13.640,2.573,0.498,0.471,2.455,1.585,12.347,0.305,0.428,0.245,...,0.201,0.240,0.302,0.067,0.333,0.14,0.315,0.166,0.216,0.231
min,17.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.00,0.000,0.000,0.000,0.000
25%,28.000,9.000,0.000,0.000,0.000,0.000,40.000,1.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.00,0.000,0.000,0.000,0.000
50%,37.000,10.000,0.000,1.000,0.000,0.000,40.000,1.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.00,0.000,0.000,0.000,0.000
75%,48.000,12.000,1.000,1.000,0.000,0.000,45.000,1.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.00,0.000,0.000,0.000,0.000
max,90.000,16.000,1.000,1.000,11.513,8.380,99.000,1.000,1.000,1.000,...,1.000,1.000,1.000,1.000,1.000,1.00,1.000,1.000,1.000,1.000


In [9]:
from sklearn.model_selection import train_test_split

# Separate features and target
X = df.drop(columns=['income'])
y = df['income']

# Train/test split (80/20, stratified to preserve class balance)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"X_train: {X_train.shape}, X_test: {X_test.shape}")
print(f"\ny_train distribution:\n{y_train.value_counts(normalize=True).round(3)}")
print(f"\ny_test distribution:\n{y_test.value_counts(normalize=True).round(3)}")

X_train: (26048, 28), X_test: (6513, 28)

y_train distribution:
income
0    0.759
1    0.241
Name: proportion, dtype: float64

y_test distribution:
income
0    0.759
1    0.241
Name: proportion, dtype: float64


In [10]:
import os

output_dir = r"C:\Users\andras.janko\Documents\CensusIncomePrediction\_data"

X_train.to_csv(os.path.join(output_dir, 'X_train.csv'), index=False)
X_test.to_csv(os.path.join(output_dir, 'X_test.csv'), index=False)
y_train.to_csv(os.path.join(output_dir, 'y_train.csv'), index=False)
y_test.to_csv(os.path.join(output_dir, 'y_test.csv'), index=False)

print("Saved files:")
for f in ['X_train.csv', 'X_test.csv', 'y_train.csv', 'y_test.csv']:
    filepath = os.path.join(output_dir, f)
    size = os.path.getsize(filepath) / 1024
    print(f"  {f}: {size:.1f} KB")

Saved files:
  X_train.csv: 1663.1 KB
  X_test.csv: 415.8 KB
  y_train.csv: 76.3 KB
  y_test.csv: 19.1 KB
